In [100]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, regularizers
from tensorflow.keras.models import Model, Sequential
from sklearn.preprocessing import MinMaxScaler

In [95]:
# Setup dataset

data = pd.read_csv("./data/data_timbre.csv")

labels = data['order']
columns_to_remove = ['Unnamed: 0', 'Unnamed: 0.1', 'key', 'mode', 'id', 'duration_ms', 'time_signature', 'track_title', 'album_title', 'album_artist', 'track_number', 'total_tracks', 'order']
data.drop(columns = columns_to_remove, axis = 1, inplace = True)

data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']] = MinMaxScaler().fit_transform(data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']])

training_set, test_set, training_labels, test_labels = train_test_split(data, labels, test_size=0.02, random_state=42)

# data
training_set

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_timbre,song_timbre_start,song_timbre_end,loudness_start,loudness_end
1220,0.740,0.574,0.797027,0.2450,0.411000,0.000000,0.2620,0.330,0.603733,0.153677,0.222035,1.000000,0.899002,0.917828
4640,0.848,0.384,0.747980,0.0617,0.067100,0.000000,0.0852,0.481,0.397013,0.171268,0.254732,0.746563,0.803116,0.343408
23,0.634,0.798,0.799489,0.0607,0.028300,0.546000,0.1000,0.494,0.503955,0.211263,0.545565,1.000000,0.810185,0.840737
297,0.669,0.743,0.653421,0.0494,0.288000,0.805000,0.3090,0.280,0.483274,0.272017,0.541325,0.404247,0.558279,0.205869
1788,0.605,0.513,0.733146,0.0337,0.774000,0.000177,0.1230,0.512,0.515948,0.142989,0.260153,1.000000,0.784888,0.893320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,0.383,0.621,0.830482,0.1130,0.027200,0.000000,0.4270,0.213,0.322708,0.261726,0.653905,1.000000,0.406734,0.688868
466,0.833,0.877,0.907398,0.2160,0.005890,0.121000,0.6690,0.038,0.515920,0.211869,0.532917,0.799374,0.560645,0.618584
3092,0.491,0.900,0.867725,0.0362,0.000323,0.036400,0.3200,0.286,0.499903,0.210321,0.462922,0.522600,0.823651,0.302241
3772,0.540,0.418,0.667308,0.0306,0.664000,0.000387,0.1060,0.284,0.543252,0.243174,0.226151,0.297094,0.681551,0.934780


In [96]:
# Setup autoencoder

encoder_layers = [
            layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9)),
            ]

decoder_layers = [
            layers.Dense(14, activation='sigmoid', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))
            ]

encoder = Sequential(encoder_layers)
decoder = Sequential(decoder_layers)

autoencoder = Sequential([encoder, decoder])

# autoencoder.compile(optimizer='adam', loss=losses.BinaryCrossentropy())
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [97]:
autoencoder.fit(training_set, training_set,
                epochs=100,
                shuffle=True,
                validation_split = 0.2)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

122/122 [==============================] - 0s 4ms/step - loss: 0.0784 - val_loss: 0.0657
Epoch 2/100
122/122 [==============================] - 0s 2ms/step - loss: 0.0564 - val_loss: 0.0490
Epoch 3/100
122/122 [==============================] - 0s 2ms/step - loss: 0.0462 - val_loss: 0.0433
Epoch 4/100
122/122 [==============================] - 0s 1ms/step - loss: 0.0426 - val_loss: 0.0409
Epoch 5/100
122/122 [==============================] - 0s 1ms/step - loss: 0.0404 - val_loss: 0.0388
Epo

In [98]:
encoded_tracks = encoder.predict(test_set)
decoded_tracks = decoder.predict(encoded_tracks)

distances = []
for i in range(len(decoded_tracks)):
    distances.append(np.linalg.norm(decoded_tracks[i]-test_set.iloc[i]))
np.mean(distances)

# autoencoder.evaluate(test_set)
# autoencoder.summary()

0.4761125606162418

In [108]:
autoencoder.save('autoencoder.h5')
encoder.save('encoder.h5')
decoder.save('decoder.h5')